# 🤖 Agent Development - Risk Analyst & Compliance Officer

Welcome to Phase 2 & 3 of the project! In this notebook, you'll develop and test both AI agents:

1. **Risk Analyst Agent** (Chain-of-Thought prompting)
2. **Compliance Officer Agent** (ReACT prompting)

## 🎯 Learning Objectives
- Implement Chain-of-Thought prompting for systematic reasoning
- Build ReACT prompting for structured action-taking
- Handle structured JSON outputs from LLMs
- Create robust error handling and validation
- Test agents with real financial data

## 🚀 Setup and Environment

In [1]:
# Import required libraries
import os
import sys
import json
import openai
from dotenv import load_dotenv
from datetime import datetime

# Add src directory to Python path for module imports
sys.path.append(os.path.abspath('../src'))

# Load environment variables
load_dotenv('../.env')

print("📚 Libraries loaded!")
print("🔐 Environment variables loaded")
print("📂 Source directory added to Python path:", os.path.abspath('../src'))

📚 Libraries loaded!
🔐 Environment variables loaded
📂 Source directory added to Python path: C:\Users\mayer\PycharmProjects\cd14685-fin-serv-agentic-c1-classroom\project\starter\src


In [2]:
# OpenAI Setup for Vocareum
import openai

# Option 1: Use the helper function from src package (recommended)
# Uncomment this after implementing foundation_sar.py:
# from src import create_vocareum_openai_client
# client = create_vocareum_openai_client()

# Option 2: Manual setup (for early development)
openai_api_key = os.getenv('OPENAI_API_KEY')

if not openai_api_key:
    print("⚠️ WARNING: No OpenAI API key found!")
    print("Please set OPENAI_API_KEY in your .env file")
    print("Get your Vocareum OpenAI API key from 'Cloud Resources' in your workspace")
else:
    # Vocareum requires routing through their servers
    client = openai.OpenAI(
        base_url="https://openai.vocareum.com/v1",
        api_key=openai_api_key
    )
    print("✅ OpenAI client initialized with Vocareum routing")
    print(f"🔑 API key: {openai_api_key[:8]}...{openai_api_key[-4:]}")
    print("📍 Base URL: https://openai.vocareum.com/v1")
    print("\n💡 Tip: After implementing foundation_sar.py, you can use:")
    print("   from src import create_vocareum_openai_client")
    print("   client = create_vocareum_openai_client()")

✅ OpenAI client initialized with Vocareum routing
🔑 API key: voc-9967...6553
📍 Base URL: https://openai.vocareum.com/v1

💡 Tip: After implementing foundation_sar.py, you can use:
   from src import create_vocareum_openai_client
   client = create_vocareum_openai_client()


## 📊 Phase 1 Review: Load Foundation Components

Before building agents, let's ensure your foundation components are working.

In [3]:
# TODO: Import your implemented foundation components
# Uncomment and modify these imports once you've implemented foundation_sar.py

from foundation_sar import (
    CustomerData,
    AccountData,
    TransactionData,
    CaseData,
    RiskAnalystOutput,
    ComplianceOfficerOutput,
    ExplainabilityLogger,
    DataLoader,
    load_csv_data
)

print("📋 TODO: Import foundation components after implementing foundation_sar.py")
print("✅ Once imported, you can create sample cases for agent testing")

📋 TODO: Import foundation components after implementing foundation_sar.py
✅ Once imported, you can create sample cases for agent testing


In [4]:
import os
import sys
import pandas as pd
import numpy as np

# Ensure src is in path
sys.path.append(os.path.abspath('../src'))

# Reload just in case
import foundation_sar
import importlib
importlib.reload(foundation_sar)

from foundation_sar import ExplainabilityLogger, DataLoader, load_csv_data

print("Starting Foundation Component Test ...")

# ---------------------------------------------------------
# 1. Load CSV data
# ---------------------------------------------------------
try:
    customers_df, accounts_df, transactions_df = load_csv_data("../data/")

    print("Applying Type-Cast Sanitization...")

    # Force Pandas to treat columns as generic objects, not specific numbers.
    # This allows them to hold "None" without converting it back to "NaN".
    customers_df = customers_df.astype(object)
    accounts_df = accounts_df.astype(object)
    transactions_df = transactions_df.astype(object)

    # Replace NaN with None.
    # Since the columns are now Objects, they will happily accept None.
    customers_df = customers_df.where(pd.notnull(customers_df), None)
    accounts_df = accounts_df.where(pd.notnull(accounts_df), None)
    transactions_df = transactions_df.where(pd.notnull(transactions_df), None)

    print(f"✅ CSV Data Loaded & Sanitized:")
    print(f"   - Customers: {len(customers_df)}")
    print(f"   - Accounts: {len(accounts_df)}")
    print(f"   - Transactions: {len(transactions_df)}")

except Exception as e:
    print(f"❌ Failed to load CSVs: {e}")
    raise e

# ---------------------------------------------------------
# 2. Create Loader
# ---------------------------------------------------------
log_path = "../outputs/audit_logs/agent_development.jsonl"
os.makedirs(os.path.dirname(log_path), exist_ok=True)
logger = ExplainabilityLogger(log_path)
loader = DataLoader(logger)

print("\nGenerating Sample Case...")

try:

    active_account_id = transactions_df['account_id'].iloc[0]
    target_cust_id = accounts_df[accounts_df['account_id'] == active_account_id]['customer_id'].iloc[0]

    print(f"   Selected Active Customer: {target_cust_id}")

    sample_customer_dict = customers_df[customers_df['customer_id'] == target_cust_id].iloc[0].to_dict()
    sample_accounts_list = accounts_df[accounts_df['customer_id'] == target_cust_id].to_dict(orient='records')

    target_acc_ids = {acc['account_id'] for acc in sample_accounts_list}
    sample_transactions_list = transactions_df[transactions_df['account_id'].isin(target_acc_ids)].to_dict(orient='records')

    test_case = loader.create_case_from_data(
        sample_customer_dict,
        sample_accounts_list,
        sample_transactions_list
    )

    print(f"Case Created Successfully")
    print(f"   Case ID: {test_case.case_id}")
    print(f"   Customer: {test_case.customer.name}")
    print(f"   Transactions: {len(test_case.transactions)}")

    # Save the case for the Agent Development step
    current_case_data = test_case

except Exception as e:
    print(f"❌ Failed to generate case: {e}")

Starting Foundation Component Test ...
Applying Type-Cast Sanitization...
✅ CSV Data Loaded & Sanitized:
   - Customers: 150
   - Accounts: 178
   - Transactions: 4268

Generating Sample Case...
   Selected Active Customer: CUST_0002
Case Created Successfully
   Case ID: 9cdd039a-5f37-4631-a350-b7a07baa89cf
   Customer: Renee Blair
   Transactions: 11


## 🔍 Phase 2: Risk Analyst Agent Development

The Risk Analyst Agent uses **Chain-of-Thought prompting** to systematically analyze suspicious activity patterns.

### 📚 Understanding Chain-of-Thought Prompting

Chain-of-Thought (CoT) prompting guides AI models through step-by-step reasoning:

1. **Explicit Steps**: Break complex reasoning into clear phases
2. **Sequential Logic**: Each step builds on previous ones
3. **Domain Expertise**: Frame AI as subject matter expert
4. **Structured Output**: Guide toward specific response format

In [5]:
# TODO: Test Chain-of-Thought prompt design
# This cell helps you design and test your CoT prompt structure

def design_cot_prompt():
    """Design and test Chain-of-Thought prompt for risk analysis"""
    
    system_prompt = """ You are a Senior Financial Crime Risk Analyst with
    0 years of experience in BSA/AML compliance. Your job is to analyze financial
    cases and identify suspicious activity.

    You must follow this 5-STEP ANALYSIS FRAMEWORK:
    1. DATA REVIEW: Scan customer profile, account types, and transaction history.
    2. PATTERN RECOGNITION: Look for structuring, round-dollar amounts, velocity spikes, or inconsistent behavior.
    3. REGULATORY MAPPING: Map observed behaviors to specific predicate offenses (e.g., structuring -> 31 CFR 1010.100).
    4. RISK QUANTIFICATION: Assess the severity (volume, frequency, impact).
    5. CLASSIFICATION DECISION: distinct classification based on evidence.

    You must output a VALID JSON object matching this schema exactly:
    {
      "classification": "Literal['Structuring', 'Sanctions', 'Fraud', 'Money_Laundering', 'Other']",
      "confidence_score": "float (0.0 to 1.0)",
      "reasoning": "Step-by-step analysis summary (max 500 chars)",
      "key_indicators": ["list", "of", "specific", "flags"],
      "risk_level": "Literal['Low', 'Medium', 'High', 'Critical']"
    }

    CRITICAL RULES:
    - If you see transactions just below $10,000, verify if it is Structuring.
    - If you see rapid movement of funds, check for Layering.
    - Be concise in your reasoning but strictly follow the evidence.
    """
    
    return system_prompt

# Test your prompt design
cot_prompt = design_cot_prompt()
print("🧠 Chain-of-Thought Prompt Design:")
print(cot_prompt[:200] + "...")
print("\n📋 TODO: Complete the prompt in risk_analyst_agent.py")

🧠 Chain-of-Thought Prompt Design:
 You are a Senior Financial Crime Risk Analyst with
    0 years of experience in BSA/AML compliance. Your job is to analyze financial
    cases and identify suspicious activity.

    You must follow t...

📋 TODO: Complete the prompt in risk_analyst_agent.py


In [6]:
# TODO: Implement and test Risk Analyst Agent - SIMPLE SMOKE TEST
# Students: Write a basic smoke test to verify your agent works

from risk_analyst_agent import RiskAnalystAgent
from foundation_sar import ExplainabilityLogger

def simple_risk_analyst_smoke_test():
    """
    STUDENT TODO: Write a simple smoke test for your Risk Analyst Agent
    
    This should be a basic test that:
    1. Creates a RiskAnalystAgent instance
    2. Creates simple test data (or uses mock data)
    3. Calls analyze_case() method
    4. Verifies the result has the expected structure
    5. Prints success/failure
    
    Keep this simple - just verify your agent doesn't crash and returns something reasonable!
    """
    print("🔍 Risk Analyst Smoke Test")
    #print("📋 TODO: Import your RiskAnalystAgent")
    #print("📋 TODO: Create agent instance: agent = RiskAnalystAgent(client, logger)")
    #print("📋 TODO: Create simple test case data")
    #print("📋 TODO: Call: result = agent.analyze_case(test_case)")
    #print("📋 TODO: Verify: result has classification, confidence_score, reasoning")
    #print("📋 TODO: Print: 'SUCCESS' or 'FAILED' with details")
    
    # Example structure (uncomment and modify when ready):
    try:
        explainability_logger = ExplainabilityLogger()
        agent = RiskAnalystAgent(client, explainability_logger)
        print("    ... agent Created")

        # Create minimal test case...
        cust = CustomerData(
            customer_id = "CUST_SMURF",
            name = "Test Smurf Subject",
            date_of_birth = "1985-05-20",
            ssn_last_4 = "2342",
            address = "123 Test St",
            customer_since = "2010-01-01",
            risk_rating = "Low"
        )
        print("    ... CustomerData Created")

        # Minimal valid Account
        acc = AccountData(
            account_id = "ACC_SMURF",
            customer_id = "CUST_SMURF",
            account_type = "Savings",
            opening_date = "2010-01-01",
            current_balance = 25000.0,
            average_monthly_balance = 5000.0,
            status = "Active"
        )
        print("    ... AccountData Created")

        # Minimal valid Transaction
        t1 = TransactionData(
            transaction_id="TX_SMURF_1",
            account_id="ACC_SMURF",
            transaction_date="2024-02-01",
            transaction_type="Deposit",
            amount=9800.0,
            method="Cash",
            description="Branch Deposit",
            location="Branch A"
        )

        # Transaction 2: Another one just below $10k on the next day
        t2 = TransactionData(
            transaction_id="TX_SMURF_2",
            account_id="ACC_SMURF",
            transaction_date="2024-02-02",
            transaction_type="Deposit",
            amount=9700.0,
            method="Cash",
            description="Branch Deposit",
            location="Branch B"
        )
        print("    ... TransactionData Created")

        # Minimal Case
        smoke_test_case = CaseData(
            case_id = "SMOKE_CASE",
            customer = cust,
            accounts = [acc],
            transactions = [t1, t2],
            case_created_at = "2024-02-03",
            data_sources = {"test": "true"}
        )
        print("    ... CaseData Created")

        # 4. Call Agent
        print("   🔄 Calling analyze_case()...")
        result = agent.analyze_case(smoke_test_case)
        print("   ✅ Result received")

        # Check Classification
        missing_fields = []

        if result.classification:
            print(f"   🏷️ Classification: {result.classification}")
        else:
            print("   ❌ ERROR: Classification is EMPTY")
            missing_fields.append("classification")

        # Check Risk Level
        if result.risk_level:
             print(f"   🚨 Risk Level:     {result.risk_level}")
        else:
             print("   ❌ ERROR: Risk Level is EMPTY")
             missing_fields.append("risk_level")

        # Check Confidence (Must be float and not None)
        if result.confidence_score is not None:
             print(f"   🎯 Confidence:     {result.confidence_score}")
        else:
             print("   ❌ ERROR: Confidence Score is MISSING")
             missing_fields.append("confidence_score")

        # Check Reasoning
        if result.reasoning and len(result.reasoning) > 10: # Ensure it's not just ".."
             print(f"   📝 Reasoning:      {result.reasoning}")
        else:
             print(f"   ❌ ERROR: Reasoning is EMPTY or too short: '{result.reasoning}'")
             missing_fields.append("reasoning")

        # --- Final Logic Assertion ---
        if missing_fields:
            print(f"\n❌ TEST FAILED: The following fields were missing: {missing_fields}")
        elif result.classification in ["Structuring", "Money_Laundering"]:
            print("\n✅ PASS: Agent correctly identified Structuring despite 'Low' customer risk.")
        else:
            print(f"\n❌ FAIL: Integrity passed, but Logic failed. Agent missed the pattern. Classified as {result.classification}.")

    except Exception as e:
        print(f"❌ ERROR: {e}")

simple_risk_analyst_smoke_test()

   ✅ Result received
   🏷️ Classification: Structuring
   🚨 Risk Level:     High
   🎯 Confidence:     0.95
   📝 Reasoning:      The customer has made two large deposits just below the $10,000 reporting threshold, indicative of potential structuring. The deposits were made at different branches, possibly to avoid detection. Despite the low risk rating, the customer's behavior is suspicious.

✅ PASS: Agent correctly identified Structuring despite 'Low' customer risk.


### 🧪 Risk Analyst Testing Framework

In [7]:
# COMPREHENSIVE Risk Analyst Testing - Import Pre-Built Test Suite
# Students: Use our comprehensive test suite instead of writing your own

import sys
import os

# Add tests directory to Python path for importing test modules
project_root = os.path.abspath('..')
tests_path = os.path.join(project_root, 'tests')
if tests_path not in sys.path:
    sys.path.insert(0, tests_path)

print(f"📁 Added tests directory to Python path: {tests_path}")

def run_comprehensive_risk_analyst_tests():
    """
    Use pre-built comprehensive test suite to validate your Risk Analyst Agent
    
    These tests validate:
    - Agent initialization and configuration
    - Case analysis with valid inputs
    - JSON parsing and error handling
    - System prompt structure and content
    - API call parameters and responses
    - Helper method functionality
    """
    print("🧪 Comprehensive Risk Analyst Testing")
    print("📋 TODO: Uncomment and run after implementing your Risk Analyst Agent")
    
    # Uncomment when your agent is ready:
    try:
        from test_risk_analyst import TestRiskAnalystAgent
        import pytest

        print("🔍 Loading comprehensive test suite...")

        # Run the test suite
        print("? Running Risk Analyst test suite...")
        result = pytest.main([
            f"{tests_path}/test_risk_analyst.py",
            "-v",
            "--tb=short"
        ])

        if result == 0:
            print("✅ All Risk Analyst tests passed!")
        else:
            print("❌ Some tests failed. Check the output above for details.")

    except ImportError as e:
        print(f"❌ Import Error: {e}")
        print("💡 Make sure you've implemented RiskAnalystAgent in src/risk_analyst_agent.py")

# Quick preview of available tests
try:
    from test_risk_analyst import TestRiskAnalystAgent
    import inspect
    
    # Get all test methods
    test_methods = [method for method in dir(TestRiskAnalystAgent) 
                   if method.startswith('test_')]
    
    print("📊 Preview of Comprehensive Risk Analyst Tests:")
    for method_name in test_methods[:5]:  # Show first 5
        method = getattr(TestRiskAnalystAgent, method_name)
        doc = method.__doc__ or method_name.replace('_', ' ').title()
        print(f"   • {doc}")
    if len(test_methods) > 5:
        print(f"   ... and {len(test_methods) - 5} more tests")
    print("\n💡 These tests validate edge cases you might not think of!")
    print("💡 Much more thorough than manual testing!")
except Exception as e:
    print(f"ℹ️ Test suite will be available after implementing foundation_sar.py: {e}")

# Call the function
run_comprehensive_risk_analyst_tests()

✅ All Risk Analyst tests passed!


## ✅ Phase 3: Compliance Officer Agent Development

The Compliance Officer Agent uses **ReACT prompting** to generate regulatory-compliant SAR narratives.

### 📚 Understanding ReACT Prompting

ReACT (Reasoning + Action) prompting separates thinking and doing:

1. **Reasoning Phase**: Analyze situation and plan approach
2. **Action Phase**: Execute specific task with informed decisions
3. **Structured Workflow**: Consistent approach to complex tasks
4. **Regulatory Compliance**: Emphasis on meeting specific requirements

In [8]:
# TODO: Test ReACT prompt design
# This cell helps you design and test your ReACT prompt structure

def design_react_prompt():
    """Design and test ReACT prompt for compliance narratives"""
    
    system_prompt = """
    You are a Senior AML Compliance Officer (ReACT Agent) for a major financial institution.
    Your task is to review a case file and the findings of a Risk Analyst, then draft a concise Suspicious Activity Report (SAR) narrative.

    # INPUT CONTEXT
    You will receive a JSON object containing:
    1. Customer & Account Details
    2. Transaction History
    3. Risk Analyst Findings (Classification, Reasoning, Risk Level)

    # ReACT FRAMEWORK INSTRUCTIONS
    You must execute this task in two phases:

    ## PHASE 1: REASONING
    Analyze the data before writing. Ask yourself:
    1. What is the specific suspicious behavior? (e.g., Structuring, Layering)
    2. Which regulations apply? (e.g., 31 CFR 1020.320 for suspicious transactions, 12 CFR 21.11)
    3. What are the "Who, What, Where, When, Why"?
    4. Are there mitigating factors? (If none, note that: "No apparent economic purpose")

    ## PHASE 2: ACTION
    Draft the narrative text strictly following these rules:
    - START with the date range and total suspicious amount.
    - BODY should detail the specific patterns (e.g., "Customer made 4 cash deposits of $9,900...").
    - CONCLUSION should state the suspicion (e.g., "Activity appears structured to evade reporting requirements").
    - TONE: Objective, dry, professional, and strictly factual.
    - LENGTH: Maximum 200 words (approx 1000 chars).
    - EXCLUDE: Personal opinions or flowery language.

    # OUTPUT FORMAT
    You must respond with a VALID JSON object matching this structure exactly:
    {
        "narrative_reasoning": "Brief explanation of your regulatory analysis (max 500 chars)",
        "regulatory_citations": ["List", "of", "Regulations"],
        "narrative": "The final SAR narrative text... (max 1000 chars)",
        "completeness_check": true
    }
    """
    
    return system_prompt

# Test your prompt design
react_prompt = design_react_prompt()
print("⚡ ReACT Prompt Design:")
print(react_prompt[:200] + "...")
print("\n📋 TODO: Complete the prompt in compliance_officer_agent.py")

⚡ ReACT Prompt Design:

    You are a Senior AML Compliance Officer (ReACT Agent) for a major financial institution.
    Your task is to review a case file and the findings of a Risk Analyst, then draft a concise Suspicious...

📋 TODO: Complete the prompt in compliance_officer_agent.py


In [9]:
# TODO: Implement and test Compliance Officer Agent - SIMPLE SMOKE TEST
# Students: Write a basic smoke test to verify your agent works

from compliance_officer_agent import ComplianceOfficerAgent
from foundation_sar import (
    ExplainabilityLogger,
    CaseData,
    CustomerData,
    AccountData,
    TransactionData,
    RiskAnalystOutput
)

def simple_compliance_officer_smoke_test():
    """
    STUDENT TODO: Write a simple smoke test for your Compliance Officer Agent
    
    This should be a basic test that:
    1. Creates a ComplianceOfficerAgent instance
    2. Creates simple test case and risk analysis data
    3. Calls generate_compliance_narrative() method  
    4. Verifies the result has a narrative with reasonable length
    5. Prints success/failure
    
    Keep this simple - just verify your agent doesn't crash and generates text!
    """
    print("✅ Compliance Officer Smoke Test")
    print("📋 TODO: Import your ComplianceOfficerAgent")
    print("📋 TODO: Create agent instance: agent = ComplianceOfficerAgent(client, logger)")
    print("📋 TODO: Create simple test case and sample risk analysis")
    print("📋 TODO: Call: result = agent.generate_compliance_narrative(case, risk_analysis)")
    print("📋 TODO: Verify: result has narrative, word count ≤ 120")
    print("📋 TODO: Print: 'SUCCESS' or 'FAILED' with details")
    
    # Example structure (uncomment and modify when ready):
    try:
    #     agent = ComplianceOfficerAgent(client, explainability_logger)
    #     # Create test case and risk analysis...
    #     result = agent.generate_compliance_narrative(test_case, test_risk_analysis)
    #     word_count = len(result.narrative.split())
    #     print(f"✅ SUCCESS: Generated {word_count} word narrative")
    #     print(f"Preview: {result.narrative[:100]}...")
    # except Exception as e:
    #     print(f"❌ FAILED: {e}")

        # Create Agent
        explainability_logger = ExplainabilityLogger()
        agent = ComplianceOfficerAgent(client, explainability_logger)
        print("    ... Agent created")

        # Create Test Data (Reusing the 'Smurfing' Scenario)
        cust = CustomerData(
            customer_id = "CUST_SMURF",
            name = "Test Smurf Subject",
            date_of_birth = "1985-05-20",
            ssn_last_4 = "2342",
            address = "123 Test St",
            customer_since = "2010-01-01",
            risk_rating = "Low"
        )
        print("    ... CustomerData created")

        acc = AccountData(
            account_id = "ACC_SMURF",
            customer_id = "CUST_SMURF",
            account_type = "Savings",
            opening_date = "2010-01-01",
            current_balance = 25000.0,
            average_monthly_balance = 5000.0,
            status = "Active"
        )
        print("    ... AccountData created")

        # Transactions: Two cash deposits just under $10k
        t1 = TransactionData(
            transaction_id = "TX1",
            account_id = "ACC_SMURF",
            transaction_date = "2024-02-01",
            transaction_type = "Deposit",
            amount = 9800.0,
            method = "Cash",
            description = "Branch Deposit",
            location = "Branch A"
        )
        print("    ... Transaction t1 created")

        t2 = TransactionData(
            transaction_id = "TX2",
            account_id = "ACC_SMURF",
            transaction_date = "2024-02-02",
            transaction_type = "Deposit",
            amount = 9700.0,
            method = "Cash",
            description = "Branch Deposit",
            location = "Branch B"
        )
        print("    ... Transaction t2 created")

        test_case = CaseData(
            case_id = "SMOKE_CO_CASE",
            customer = cust,
            accounts = [acc],
            transactions = [t1, t2],
            case_created_at = "2024-02-03",
            data_sources = {"test": "true"}
        )
        print("    ... CaseData created")

        # Create Mock Risk Analysis (Simulating the output from the previous agent)
        test_risk_analysis = RiskAnalystOutput(
            classification = "Structuring",
            confidence_score = 0.95,
            reasoning = "Patient executed two large cash deposits ($9,800 and $9,700) on consecutive days at different branches, clearly avoiding the $10,000 CTR threshold.",
            key_indicators = ["threshold avoidance", "multi-branch activity", "cash deposits"],
            risk_level = "High"
        )
        print("    ... RiskAnalystOutput (Mock) created")


        print("   🔄 Calling generate_compliance_narrative()...")
        result = agent.generate_compliance_narrative(test_case, test_risk_analysis)
        print("   ✅ Compliance Officer Agent's Result received")

        # Verify Outputs - strict checks
        errors = []

        # --- Check 1: Narrative (Max 1000 chars, Max 120 words) ---
        if not result.narrative:
            print("   ❌ ERROR: Narrative is EMPTY")
            errors.append("narrative_empty")
        else:
            char_count = len(result.narrative)
            word_count = len(result.narrative.split())
            print(f"   📝 Narrative: {word_count} words | {char_count} chars")
            print(f"      \"{result.narrative[:100]}...\"")

            if char_count > 1000:
                print(f"   ❌ ERROR: Narrative too long ({char_count} > 1000 chars)")
                errors.append("narrative_char_limit")

            if word_count > 120:
                print(f"   ❌ ERROR: Narrative exceeds word limit ({word_count} > 120 words)")
                errors.append("narrative_word_limit")

        # --- Check 2: Narrative Reasoning (Max 500 chars) ---
        if not result.narrative_reasoning:
            print("   ❌ ERROR: Narrative Reasoning is EMPTY")
            errors.append("narrative_reasoning_empty")
        else:
            reasoning_len = len(result.narrative_reasoning)
            print(f"   🧠 Reasoning: {reasoning_len} chars")

            if reasoning_len > 500:
                print(f"   ❌ ERROR: Reasoning too long ({reasoning_len} > 500 chars)")
                errors.append("reasoning_char_limit")

        # --- Check 3: Regulatory Citations ---
        if result.regulatory_citations and len(result.regulatory_citations) > 0:
            print(f"   ⚖️ Citations: {result.regulatory_citations}")
        else:
            print("   ❌ ERROR: No regulatory citations found")
            errors.append("regulatory_citations_empty")

        # --- Check 4: Completeness Check ---
        if result.completeness_check is True:
             print("   ✅ Completeness Check: PASS")
        else:
             print(f"   ❌ ERROR: Completeness check failed (Value: {result.completeness_check})")
             errors.append("completeness_check_failed")

        # --- Final Verdict ---
        if errors:
            print(f"\n❌ TEST FAILED: The following checks failed: {errors}")
        else:
            print(f"\n✅ PASS: Compliance Officer generated a valid narrative meeting all strict criteria.")

    except Exception as e:
        print(f"❌ ERROR: {e}")

simple_compliance_officer_smoke_test()

   ✅ Compliance Officer Agent's Result received
   📝 Narrative: 49 words | 325 chars
      "Between 2024-02-01 and 2024-02-02, Test Smurf Subject (ID: CUST_SMURF) deposited a total of $19,500 ..."
   🧠 Reasoning: 284 chars
   ⚖️ Citations: ['31 CFR 1020.320']
   ✅ Completeness Check: PASS

✅ PASS: Compliance Officer generated a valid narrative meeting all strict criteria.


### 🧪 Compliance Officer Testing Framework

In [15]:
# COMPREHENSIVE Compliance Officer Testing - Import Pre-Built Test Suite
# Students: Use our comprehensive test suite instead of writing your own

import sys
import os

# Add tests directory to Python path for importing test modules (if not already added)
project_root = os.path.abspath('..')
tests_path = os.path.join(project_root, 'tests')
if tests_path not in sys.path:
    sys.path.insert(0, tests_path)

print(f"📁 Tests directory in Python path: {tests_path}")

def run_comprehensive_compliance_officer_tests():
    """
    Use pre-built comprehensive test suite to validate your Compliance Officer Agent
    
    These tests validate:
    - Agent initialization and configuration
    - Narrative generation with valid inputs
    - Word count limits (≤120 words)
    - Regulatory citations inclusion
    - JSON parsing and error handling
    - ReACT prompt structure validation
    """
    print("🧪 Comprehensive Compliance Officer Testing")
    print("📋 TODO: Uncomment and run after implementing your Compliance Officer Agent")
    
    # Uncomment when your agent is ready:
    try:
        from test_compliance_officer import TestComplianceOfficerAgent
        import pytest

        print("? Loading comprehensive test suite...")

        # Run the test suite
        print("🚀 Running Compliance Officer test suite...")
        result = pytest.main([
            f"{tests_path}/test_compliance_officer.py",
            "-v",
            "--tb=short"
        ])

        if result == 0:
            print("✅ All Compliance Officer tests passed!")
        else:
            print("❌ Some tests failed. Check the output above for details.")

    except ImportError as e:
        print(f"❌ Import Error: {e}")
        print("💡 Make sure you've implemented ComplianceOfficerAgent in src/compliance_officer_agent.py")

# Quick preview of available tests
try:
    from test_compliance_officer import TestComplianceOfficerAgent
    import inspect
    
    # Get all test methods
    test_methods = [method for method in dir(TestComplianceOfficerAgent) 
                   if method.startswith('test_')]
    
    print("📝 Preview of Comprehensive Compliance Officer Tests:")
    for method_name in test_methods[:5]:  # Show first 5
        method = getattr(TestComplianceOfficerAgent, method_name)
        doc = method.__doc__ or method_name.replace('_', ' ').title()
        print(f"   • {doc}")
    if len(test_methods) > 5:
        print(f"   ... and {len(test_methods) - 5} more tests")
    print("\n💡 These tests validate regulatory compliance requirements!")
    print("💡 Includes word limits, citations, and required elements!")
except Exception as e:
    print(f"ℹ️ Test suite will be available after implementing foundation_sar.py: {e}")

# Call the function
run_comprehensive_compliance_officer_tests()

✅ All Compliance Officer tests passed!


In [14]:
# COMPLETE AGENT TESTING - Two-Tier Approach
# Students: Use this to test both agents together

def complete_agent_testing_workflow():
    """
    Complete testing workflow using two-tier approach:
    
    TIER 1: Simple Smoke Tests (You write these)
    - Basic functionality verification
    - Quick sanity checks
    - Development debugging
    
    TIER 2: Comprehensive Test Suites (Pre-built for you)
    - Complex edge cases
    - Regulatory compliance validation
    - Professional-grade testing
    """
    print("🔬 Complete Agent Testing Workflow")
    print("=" * 50)
    
    print("\n📋 TIER 1: Simple Smoke Tests (DO FIRST)")
    print("   1. Write simple_risk_analyst_smoke_test() - verify basic functionality")
    print("   2. Write simple_compliance_officer_smoke_test() - verify basic functionality")
    print("   3. Fix any basic issues before moving to Tier 2")
    
    print("\n🧪 TIER 2: Comprehensive Test Suites (DO AFTER TIER 1 PASSES)")
    print("   1. Run comprehensive risk analyst test suite (10 comprehensive tests)")
    print("   2. Run comprehensive compliance officer test suite (10 comprehensive tests)")
    print("   3. Get detailed pass/fail results with specific feedback")
    
    print("\n💡 WHY THIS APPROACH?")
    print("   ✅ Tier 1: Quick feedback while developing")
    print("   ✅ Tier 2: Professional validation without writing complex tests")
    print("   ✅ Saves time: You focus on implementation, not test creation")
    print("   ✅ Better coverage: Our test suites test edge cases you might miss")

# Quick test runner when both agents are ready
def run_both_agents_quick_test():
    """Quick test of both agents using pre-built test suites"""
    print("🚀 Quick Test of Both Agents")
    print("📋 TODO: Uncomment when both agents are implemented")
    
    #Uncomment when ready:
    try:
        import pytest

        print("🔍 Running quick tests for both agents...")

        # Run a subset of tests for quick validation
        risk_result = pytest.main([
            f"{tests_path}/test_risk_analyst.py::TestRiskAnalystAgent::test_agent_initialization",
            f"{tests_path}/test_risk_analyst.py::TestRiskAnalystAgent::test_analyze_case_success",
            "-v"
        ])

        compliance_result = pytest.main([
            f"{tests_path}/test_compliance_officer.py::TestComplianceOfficerAgent::test_agent_initialization",
            f"{tests_path}/test_compliance_officer.py::TestComplianceOfficerAgent::test_generate_compliance_narrative_success",
            "-v"
        ])

        if risk_result == 0 and compliance_result == 0:
            print("🎉 Both agents working! Ready for full test suite testing!")
        else:
            print("⚠️ Fix issues before running comprehensive tests")
            if risk_result != 0:
                print("   🔍 Risk Analyst needs fixes")
            if compliance_result != 0:
                print("   📝 Compliance Officer needs fixes")

    except ImportError as e:
        print(f"❌ Import Error: {e}")
        print("💡 Make sure both agents are implemented")

complete_agent_testing_workflow()
run_both_agents_quick_test()



============================== warnings summary ===============================
..\..\..\.venv\Lib\site-packages\_pytest\config\__init__.py:1303
  C:\Users\mayer\PycharmProjects\cd14685-fin-serv-agentic-c1-classroom\.venv\Lib\site-packages\_pytest\config\__init__.py:1303: PytestAssertRewriteWarning: Module already imported so cannot be rewritten; anyio
    self._mark_plugins_for_rewrite(hook, disable_autoload)

..\..\..\..\..\..\..\Python314\Lib\unittest\mock.py:1170
  C:\Python314\Lib\unittest\mock.py:1170: PytestCollectionWarning: cannot collect 'test_client' because it is not a function.
    def __call__(self, /, *args, **kwargs):

..\..\..\..\..\..\..\Python314\Lib\unittest\mock.py:1170
  C:\Python314\Lib\unittest\mock.py:1170: PytestCollectionWarning: cannot collect 'test_logger' because it is not a function.
    def __call__(self, /, *args, **kwargs):

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
======================== 2 passed, 3 warnings in 0.02s 

## 🔗 Phase 4 Preview: Agent Integration

Once both agents are working, you'll integrate them into a complete workflow.

In [13]:
# TODO: Preview of integrated workflow
# This will be fully implemented in the next notebook

def preview_integrated_workflow():
    """Preview of how agents will work together"""
    
    workflow_steps = [
        "1. 📊 Load and validate case data",
        "2. 🔍 Risk Analyst performs Chain-of-Thought analysis",
        "3. 👤 Human review and approval gate",
        "4. ✅ Compliance Officer generates ReACT narrative (if approved)",
        "5. 📄 Generate complete SAR document",
        "6. 📊 Log audit trail and efficiency metrics"
    ]
    
    print("🔗 Integrated SAR Processing Workflow:")
    for step in workflow_steps:
        print(step)
    
    print("\n💡 Key Benefits:")
    print("• Two-stage processing reduces AI costs")
    print("• Human oversight ensures regulatory compliance")
    print("• Complete audit trails for examination")
    print("• Standardized analytical approaches")

preview_integrated_workflow()

🔗 Integrated SAR Processing Workflow:
1. 📊 Load and validate case data
2. 🔍 Risk Analyst performs Chain-of-Thought analysis
3. 👤 Human review and approval gate
4. ✅ Compliance Officer generates ReACT narrative (if approved)
5. 📄 Generate complete SAR document
6. 📊 Log audit trail and efficiency metrics

💡 Key Benefits:
• Two-stage processing reduces AI costs
• Human oversight ensures regulatory compliance
• Complete audit trails for examination
• Standardized analytical approaches


## 📝 Development Checklist - Two-Tier Testing Approach

### ✅ Risk Analyst Agent (Phase 2)
- [x] Implement Chain-of-Thought system prompt
- [x] Create `analyze_case` method with error handling
- [x] Add JSON parsing and validation
- [x] **TIER 1**: Write simple smoke test (verify basic functionality)
- [x] **TIER 2**: Run comprehensive pre-built test suite (10 comprehensive tests)
- [x] Fix any issues identified by test suite

### ✅ Compliance Officer Agent (Phase 3)  
- [x] Implement ReACT system prompt
- [x] Create `generate_compliance_narrative` method
- [x] Add narrative validation (word count, terminology)
- [x] **TIER 1**: Write simple smoke test (verify basic functionality)
- [x] **TIER 2**: Run comprehensive pre-built test suite (10 comprehensive tests)
- [x] Fix any issues identified by test suite

### ✅ Testing Strategy Benefits
- [x] **Time Savings**: Focus on implementation, not complex test creation
- [x] **Better Coverage**: Pre-built test suites test edge cases you might miss
- [x] **Quick Feedback**: Simple smoke tests for rapid development cycles
- [x] **Professional Validation**: Comprehensive test suites ensure production readiness
- [x] **Regulatory Compliance**: Built-in checks for SAR requirements

### 💡 **Testing Workflow**
1. **Start with Tier 1**: Write simple smoke tests to verify your agents don't crash
2. **Fix basic issues**: Iterate quickly with simple tests during development
3. **Move to Tier 2**: Run comprehensive test suites when basic functionality works
4. **Analyze results**: Use detailed feedback to improve agent performance
5. **Iterate**: Refine prompts and logic based on test results

## 🚀 Next Steps

1. **Complete Agent Implementation**: Finish both agent classes in the src/ directory
2. **Run Two-Tier Testing**: Start with smoke tests, then comprehensive test suites
3. **Workflow Integration**: Move to the next notebook for complete system integration
4. **Human-in-the-Loop**: Implement decision gates and review processes

## 📊 Available Test Suites Summary

**Risk Analyst Test Suite (10 tests):**
- Agent initialization and configuration
- Case analysis with valid JSON responses
- JSON parsing and error handling
- System prompt structure validation
- API call parameter verification
- Helper method functionality
- Edge case handling

**Compliance Officer Test Suite (10 tests):**
- Agent initialization and configuration
- Narrative generation with valid responses
- Word count validation (≤120 words)
- Regulatory citations inclusion
- JSON parsing and error handling
- ReACT prompt structure validation
- API call parameter verification

**Ready to build intelligent agents with professional-grade testing! 🕵️‍♀️**